In [1]:
import pandas as pd
import numpy as np
from datetime import date, timedelta, datetime
from gensim.models import Word2Vec, KeyedVectors
import string
import pickle

In [2]:
# import google's word2vec located: https://drive.google.com/file/d/0B7XkCwpI5KDYNlNUTTlSS21pQmM/edit?usp=sharing
w2v_model = KeyedVectors.load_word2vec_format('../../Downloads/GoogleNews-vectors-negative300.bin', binary=True)  

In [3]:
ticker = 'TSLA'

In [34]:
df = pd.read_csv(f'./data/{ticker}.csv')

In [6]:
def clean2vect(articles):
    new_list = []
    for a in articles:
        new_list.append(a.replace(',','').translate(string.punctuation))
    vects = []
    for a in new_list:
        vector = []
        for word in a:
            try:
                vector.append(w2v_model[word])
            except:
                continue
        vects.append(sum(vector)/len(vector))
    return vects

#### *NOTE Do we need to scale vectors?

In [35]:
df['article'] = clean2vect(df['article'])
df['comments'] = clean2vect(df['comments'])
df['name'] = clean2vect(df['name'])

df['comments_qty'] = [int(i.split(" ")[0]) for i in df['comments_qty']]

### Set Article DataFrame to Date Index to match with stock info

In [36]:
df['date'][0] = 'Sat, Jun. 27'

/Users/danielgurzi/opt/anaconda3/envs/dsi/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [37]:
df['date'][0]

'Sat, Jun. 27'

In [27]:
def art_date(df):
    dates = []
    for i in df.date:
        if "Today" in i:
            dates.append('2020-' + str(i.replace(i, date.today().strftime("%m-%d"))))
        elif "Yesterday" in i:
            dates.append('2020-' + str(i.replace(i, (date.today() - timedelta(days = 1)).strftime("%m-%d"))))
        else:
            dates.append('2020-' + str(datetime.strptime(i,'%a, %b. %d').strftime('%m-%d')))
#         set dataframe to new dates columns as index
    df.date = dates
    df.date = pd.to_datetime(df['date'])
    df.set_index('date', inplace=True)
    df.sort_index(inplace=True)
    return df

In [38]:
df = art_date(df)

In [39]:
df

,href,name,comments_qty,article,comments
date,,,,,
2020-06-17,https://seekingalpha.com/article/4354413-tesla...,"[-0.1251831, 0.0914804, 0.037174933, 0.1329040...",664,"[-0.1630917, 0.114122, -0.003956805, 0.1314682...","[-0.117103055, 0.073532425, 0.010909792, 0.118..."
2020-06-17,https://seekingalpha.com/article/4354297-why-t...,"[-0.17534436, 0.10613291, 0.043029785, 0.11086...",643,"[-0.1708225, 0.11009591, -0.004753023, 0.13495...","[-0.11660118, 0.07233955, 0.008581403, 0.11385..."
2020-06-24,https://seekingalpha.com/article/4355293-tesla...,"[-0.14043659, 0.12997104, 0.027858568, 0.09032...",526,"[-0.17009178, 0.11974242, -0.007894693, 0.1358...","[-0.10882322, 0.070975244, 0.017579183, 0.1148..."
2020-06-25,https://seekingalpha.com/article/4355755-tesla...,"[-0.16154553, 0.0910957, 0.030680338, 0.131354...",233,"[-0.17078684, 0.11602321, -0.004853303, 0.1307...","[-0.14052476062695726, 0.09001996759573175, 0...."
2020-06-25,https://seekingalpha.com/article/4355742-quest...,"[-0.16118637, 0.109375, 0.0148103675, 0.119847...",115,"[-0.16996387, 0.11208412, -0.0021054663, 0.132...","[-0.14066736, 0.08786906, 0.0010140183, 0.1290..."
2020-06-25,https://seekingalpha.com/article/4355640-tesla...,"[-0.13049117, 0.1305417, 0.030739706, 0.091169...",481,"[-0.16234204, 0.10811482, -0.005894934, 0.1314...","[-0.117103055, 0.073532425, 0.010909792, 0.118..."
2020-06-27,https://seekingalpha.com/article/4355962-stock...,"[-0.1537897, 0.08553932, 0.026596796, 0.140951...",92,"[-0.15305372, 0.10253333, 0.011405529, 0.12647...","[-0.13690521, 0.08726364, 0.00089071837, 0.130..."
2020-06-30,https://seekingalpha.com/article/4355897-tesla...,"[-0.1558468, 0.09118097, 0.03433875, 0.1200589...",173,"[-0.13890474, 0.0902732, 0.00077491574, 0.1269...","[-0.13601495, 0.08752184, 0.0019295756, 0.1264..."
2020-06-30,https://seekingalpha.com/news/3586613-bezos-co...,"[-0.15639596, 0.09136698, 0.012404732, 0.12507...",197,"[-0.16426292, 0.100805014, 0.0024576695, 0.130...","[-0.100484215, 0.10232205, 0.003648546, 0.1076..."


In [40]:
df2 = df.groupby(['date'])[['name','article','comments']].sum()
df2[['comments_qty']] = df.groupby(['date'])[['comments_qty']].sum()

In [41]:
df2

,name,article,comments,comments_qty
date,,,,
2020-06-17,"[-0.30052745, 0.1976133, 0.08020472, 0.2437703...","[-0.33391422, 0.22421792, -0.008709827, 0.2664...","[-0.23370424, 0.14587197, 0.019491196, 0.23246...",1307
2020-06-24,"[-0.14043659, 0.12997104, 0.027858568, 0.09032...","[-0.17009178, 0.11974242, -0.007894693, 0.1358...","[-0.10882322, 0.070975244, 0.017579183, 0.1148...",526
2020-06-25,"[-0.45322308, 0.3310124, 0.07623041, 0.3423712...","[-0.50309277, 0.33622217, -0.012853703, 0.3943...","[-0.39829517974774586, 0.2514214554826374, 0.0...",829
2020-06-27,"[-0.1537897, 0.08553932, 0.026596796, 0.140951...","[-0.15305372, 0.10253333, 0.011405529, 0.12647...","[-0.13690521, 0.08726364, 0.00089071837, 0.130...",92
2020-06-30,"[-0.31224275, 0.18254796, 0.046743482, 0.24513...","[-0.30316764, 0.19107822, 0.0032325853, 0.2572...","[-0.23649916, 0.1898439, 0.0055781216, 0.23408...",370


In [45]:
dbfile = open('articlePickle3', 'ab') 
pickle.dump(df2, dbfile)                      
dbfile.close() 